# 6. Tuning

In [1]:
MODEL = 'DNNc'
TUNING_MODEL = 'jiaocheng'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [4]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [ ]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [ ]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [ ]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [ ]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [ ]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [ ]:
import JiaoCheng_H2H

In [ ]:
jiaocheng = JiaoCheng_H2H.JiaoCheng()

In [ ]:
from models.DenseNeuralNetwork import DNN_const_sk as DNN

In [ ]:
jiaocheng.read_in_model(DNN, 'Regression')

In [ ]:
jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

In [ ]:
parameter_choices = {
    'n_hidden_layers': (1, 2, 3, 4, 5),
    'alpha': (0.00000001, 0.000001, 0.0001, 0.01, 0.1),
    'learning_rate_init': (0.0001, 0.001, 0.01, 0.1),
    'activation': ('relu', 'logistic'),
    'hidden_layer_n_neurons': (10, 20, 30, 40, 50, 60)

}

In [ ]:
jiaocheng.set_hyperparameters(parameter_choices)

In [ ]:
jiaocheng.set_non_tuneable_hyperparameters({'random_state': 19260817, 'max_iter': 1000, 'learning_rate': 'adaptive', 'batch_size': 'auto'})

In [ ]:
jiaocheng.set_features(ningxiang_output)

In [ ]:
jiaocheng.set_tuning_order(['features', 'activation', 'hidden_layer_n_neurons', 'n_hidden_layers', 'alpha', 'learning_rate_init'])

In [ ]:
jiaocheng.set_hyperparameter_default_values({
    'features': 0,
    'activation': 'logistic',
    'n_hidden_layers': 1,
    'alpha': 0.0001,
    'learning_rate_init': 0.001,
    'hidden_layer_n_neurons': 30}
  )

In [ ]:
try:
  jiaocheng.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')
  print('True')
except:
  print('False')

In [ ]:
jiaocheng.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

In [ ]:
jiaocheng.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

In [ ]:
jiaocheng.train_other_info = train_other_info
jiaocheng.val_other_info = val_other_info
jiaocheng.test_other_info = test_other_info

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
jiaocheng.tune()

In [ ]:
jiaocheng.view_best_combo_and_score()